<span style="color:#1B65F1;font-weight:600;font-size:30px"> 
CloudWatch Log Retrieval
</span> <br>

The following notebook assists with identifying relevant CloudWatch log items related to our 25MB Parallel Inference run. 
- See second section for data pull from all parallel execution logs. 

In [1]:
import numpy as np
import pandas as pd
import math
from datetime import datetime
pd.set_option('max_colwidth', 300)

<span style="color:#1B65F1;font-weight:550;font-size:26px"> 
Result Retrievel for a Single File Review
</span> <br>
Use the following code to capture all results related to a single WS.

In [54]:
## Load in Log
# step_function_logdata = pd.read_csv('Cosmic_AI_Logs/Trial_Two_logs-insights-results.csv')
# filename_prefix = '25_10'
# parallel_logdata = pd.read_csv('Cosmic_AI_Logs/cw_log_25mb_10.csv')
filename_prefix = '25_80'
parallel_logdata = pd.read_csv('Cosmic_AI_Logs/cw_log_25mb_80.csv')

In [55]:
## Review data
print(parallel_logdata.shape)
parallel_logdata[0:2]

(2960, 4)


,@timestamp,@message,@logStream,@log
0,2024-11-26 18:11:25.356,END RequestId: b592ae7f-6e4e-4b39-9791-4bb4c485f7b0\n,2024/11/26/[$LATEST]c83844236cb3452c8030b870dc484b1e,211125778552:/aws/lambda/cosmic-executor
1,2024-11-26 18:11:25.356,REPORT RequestId: b592ae7f-6e4e-4b39-9791-4bb4c485f7b0\tDuration: 14547.75 ms\tBilled Duration: 15679 ms\tMemory Size: 10240 MB\tMax Memory Used: 2878 MB\tInit Duration: 1131.19 ms\t\n,2024/11/26/[$LATEST]c83844236cb3452c8030b870dc484b1e,211125778552:/aws/lambda/cosmic-executor


### Search Based on Request ID

In [56]:
single_para_filter = parallel_logdata[
                    parallel_logdata['@message'].str.contains("REPORT RequestId", na=False) ]
single_para_filter[0:10]

,@timestamp,@message,@logStream,@log
1,2024-11-26 18:11:25.356,REPORT RequestId: b592ae7f-6e4e-4b39-9791-4bb4c485f7b0\tDuration: 14547.75 ms\tBilled Duration: 15679 ms\tMemory Size: 10240 MB\tMax Memory Used: 2878 MB\tInit Duration: 1131.19 ms\t\n,2024/11/26/[$LATEST]c83844236cb3452c8030b870dc484b1e,211125778552:/aws/lambda/cosmic-executor
11,2024-11-26 18:11:23.982,REPORT RequestId: 113e106c-e8cd-45c2-9f8c-b1dc01b50b17\tDuration: 13209.44 ms\tBilled Duration: 14290 ms\tMemory Size: 10240 MB\tMax Memory Used: 2848 MB\tInit Duration: 1080.31 ms\t\n,2024/11/26/[$LATEST]8cc57e8c03904fdb98c8d3d0b36dda8c,211125778552:/aws/lambda/cosmic-executor
15,2024-11-26 18:11:23.896,REPORT RequestId: 44fd4add-bd7b-4323-9be0-3bda3cfaf74f\tDuration: 13073.13 ms\tBilled Duration: 14142 ms\tMemory Size: 10240 MB\tMax Memory Used: 2767 MB\tInit Duration: 1068.02 ms\t\n,2024/11/26/[$LATEST]c6d1714f13074982829e3df81f537aab,211125778552:/aws/lambda/cosmic-executor
18,2024-11-26 18:11:23.788,REPORT RequestId: 7f31cc67-aff6-4a18-96aa-80517bb69fba\tDuration: 12884.53 ms\tBilled Duration: 14109 ms\tMemory Size: 10240 MB\tMax Memory Used: 2842 MB\tInit Duration: 1223.88 ms\t\n,2024/11/26/[$LATEST]770271f7690d42d084ecb50ec8dd3385,211125778552:/aws/lambda/cosmic-executor
21,2024-11-26 18:11:23.786,REPORT RequestId: a73fae8e-0c78-40f7-af29-c91b3636b4da\tDuration: 12891.80 ms\tBilled Duration: 14037 ms\tMemory Size: 10240 MB\tMax Memory Used: 2840 MB\tInit Duration: 1144.88 ms\t\n,2024/11/26/[$LATEST]65808dcf90f84e8b9e96e44c2da72ef3,211125778552:/aws/lambda/cosmic-executor
...,...,...,...,...
738,2024-11-26 18:11:20.662,REPORT RequestId: f985338e-852b-4ef0-807a-4b465ca0e110\tDuration: 10265.30 ms\tBilled Duration: 10964 ms\tMemory Size: 10240 MB\tMax Memory Used: 2866 MB\tInit Duration: 697.75 ms\t\n,2024/11/26/[$LATEST]dd6f4be57b1b4e898f0a2d0b5bc79de7,211125778552:/aws/lambda/cosmic-executor
749,2024-11-26 18:11:20.599,REPORT RequestId: d3212763-4e5f-4aad-97e6-5f8d1b6e2276\tDuration: 10285.44 ms\tBilled Duration: 11001 ms\tMemory Size: 10240 MB\tMax Memory Used: 2849 MB\tInit Duration: 715.09 ms\t\n,2024/11/26/[$LATEST]e3103c639d114d27a56d640f84c54c95,211125778552:/aws/lambda/cosmic-executor
776,2024-11-26 18:11:20.503,REPORT RequestId: f36c7757-06bf-4a11-ac27-9984b812bd9a\tDuration: 10153.26 ms\tBilled Duration: 10874 ms\tMemory Size: 10240 MB\tMax Memory Used: 2873 MB\tInit Duration: 720.23 ms\t\n,2024/11/26/[$LATEST]5c72aa9344d049caa93f0c4f0ae398bf,211125778552:/aws/lambda/cosmic-executor
787,2024-11-26 18:11:20.376,REPORT RequestId: 6ef53fa9-2803-48a7-a6f6-b267b4955e80\tDuration: 9992.90 ms\tBilled Duration: 10748 ms\tMemory Size: 10240 MB\tMax Memory Used: 2856 MB\tInit Duration: 754.83 ms\t\n,2024/11/26/[$LATEST]ca14acafd2854428a31f4bce4aee9121,211125778552:/aws/lambda/cosmic-executor


In [36]:
run_index_vals = single_para_filter.index; print(run_index_vals)

Index([1, 12, 15, 18, 21, 24, 27, 38, 41, 76], dtype='int64')


<div style="border-top: 8px solid #16145b; margin-top: 10px; margin-bottom: 10px;"></div>

<span style="color:#1B65F1;font-weight:550;font-size:26px"> 
Result Retrievel from Multiple Executions
</span> <br>
Use the following code to capture all results related to a single WS.

In [169]:
### One Time set
all_parallel_runs_df = pd.DataFrame()

In [184]:
## Load in Log
# filename_prefix = '25_10'
# parallel_logdata = pd.read_csv('Cosmic_AI_Logs/cw_log_25mb_10.csv')
# filename_prefix = '25_40'
# parallel_logdata = pd.read_csv('Cosmic_AI_Logs/cw_log_25mb_40.csv')
# filename_prefix = '25_80'
# parallel_logdata = pd.read_csv('Cosmic_AI_Logs/cw_log_25mb_80.csv')
filename_prefix = '25_120'
parallel_logdata = pd.read_csv('Cosmic_AI_Logs/cw_log_25mb_120.csv')

In [185]:
dataframes_list = []

In [186]:

para_logs_filter = parallel_logdata[
                    parallel_logdata['@message'].str.contains("REPORT RequestId", na=False) ]

df_index_vals = para_logs_filter.index; print(df_index_vals)
    
for ii in df_index_vals:
    print(f' Received Index in run: {ii}')
    target_text = para_logs_filter.loc[ii,'@message']  ; ### print(received_text) 
    
    ## Grab Data
    duration_met = target_text.split("Duration:")[1].split()[0]
    print(duration_met)
    bill_duration_met = target_text.split("Billed Duration:")[1].split()[0]
    memory_met = target_text.split("Memory Size:")[1].split()[0]
    max_memory_met = target_text.split("Max Memory Used:")[1].split()[0]
    init_duration_met = target_text.split("Init Duration:")[1].split()[0]
    
    filename_new = filename_prefix + '_'+ str(ii)
    data_temp = {'paralell_run': filename_prefix, 'id': filename_new,  
                 'duration': duration_met, 'billed duration': bill_duration_met,
                'memory size':memory_met, 'max memory used': max_memory_met,
                'init duration': init_duration_met}
      
    results_temp_df = pd.DataFrame([data_temp])  # Wrap the dictionary in a list
    dataframes_list.append(results_temp_df)  # Append the DataFrame to the list

# Combine all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes_list, ignore_index=False)

Index([   3,   14,   17,   20,   47,   50,   61,   80,   83,   86,
       ...
       1141, 1144, 1179, 1190, 1201, 1212, 1215, 1218, 1221, 1223],
      dtype='int64', length=120)
 Received Index in run: 3
22077.12
 Received Index in run: 14
21011.03
 Received Index in run: 17
20639.67
 Received Index in run: 20
20196.31
 Received Index in run: 47
17601.34
 Received Index in run: 50
16436.85
 Received Index in run: 61
16183.07
 Received Index in run: 80
14989.68
 Received Index in run: 83
14806.14
 Received Index in run: 86
14688.54
 Received Index in run: 89
13884.06
 Received Index in run: 92
14417.75
 Received Index in run: 95
13736.44
 Received Index in run: 114
13513.70
 Received Index in run: 125
13197.59
 Received Index in run: 136
12860.50
 Received Index in run: 139
13431.52
 Received Index in run: 142
12989.50
 Received Index in run: 145
13282.22
 Received Index in run: 164
13267.84
 Received Index in run: 167
12643.86
 Received Index in run: 170
13178.32
 Received Index in ru

In [187]:
print(combined_df.shape)
combined_df = combined_df.reset_index()
combined_df

(120, 7)


,index,paralell_run,id,duration,billed duration,memory size,max memory used,init duration
0,0,25_120,25_120_3,22077.12,23191,10240,2777,1113.76
1,0,25_120,25_120_14,21011.03,22003,10240,2864,991.14
2,0,25_120,25_120_17,20639.67,21704,10240,2768,1064.10
3,0,25_120,25_120_20,20196.31,21067,10240,2843,870.41
4,0,25_120,25_120_47,17601.34,18258,10240,2534,656.07
...,...,...,...,...,...,...,...,...
115,0,25_120,25_120_1212,10208.00,10859,10240,2849,650.58
116,0,25_120,25_120_1215,10201.27,10901,10240,2875,699.43
117,0,25_120,25_120_1218,10218.80,10884,10240,2848,664.78
118,0,25_120,25_120_1221,9741.73,10460,10240,2854,718.02


### Combine DF Results

In [188]:
## Save to new DF after each folder load
# result = pd.concat([df1, df2], axis=1)
all_parallel_runs_df = pd.concat([all_parallel_runs_df, combined_df], axis=0, ignore_index=False)
all_parallel_runs_df.shape


(250, 8)

In [189]:
columns_to_convert = ['duration', 'billed duration', 'memory size', 'max memory used', 'init duration']
all_parallel_runs_df[columns_to_convert] = all_parallel_runs_df[columns_to_convert].astype(float)
all_parallel_runs_df.head()

,index,paralell_run,id,duration,billed duration,memory size,max memory used,init duration
0,0,25_10,25_10_1,12606.91,13836.0,10240.0,2865.0,1228.99
1,0,25_10,25_10_12,11988.23,12906.0,10240.0,2837.0,917.59
2,0,25_10,25_10_15,11855.42,12796.0,10240.0,2852.0,939.67
3,0,25_10,25_10_18,11571.60,12602.0,10240.0,2820.0,1029.70
4,0,25_10,25_10_21,11782.33,12574.0,10240.0,2850.0,791.44


<div style="border-top: 8px solid #16145b; margin-top: 10px; margin-bottom: 10px;"></div>

## Aggregation
- See Result Dataframes for final results from all parallel executions consolidated. 

In [203]:
agg_df = all_parallel_runs_df.copy()
agg_df.drop(['index', 'id'], axis=1, inplace=True)
print(agg_df.shape)
agg_df.head()           

(250, 6)


,paralell_run,duration,billed duration,memory size,max memory used,init duration
0,25_10,12606.91,13836.0,10240.0,2865.0,1228.99
1,25_10,11988.23,12906.0,10240.0,2837.0,917.59
2,25_10,11855.42,12796.0,10240.0,2852.0,939.67
3,25_10,11571.60,12602.0,10240.0,2820.0,1029.70
4,25_10,11782.33,12574.0,10240.0,2850.0,791.44


In [204]:
result_group = agg_df.groupby('paralell_run').mean()
result_group = result_group.sort_values(by="duration", ascending=True)  # Sort in ascending order
result_group

,duration,billed duration,memory size,max memory used,init duration
paralell_run,,,,,
25_40,11268.680000,12113.9750,10240.0,2823.475000,844.840250
25_10,11480.164000,12355.2000,10240.0,2841.000000,874.620000
25_80,11666.122375,12644.7625,10240.0,2803.412500,978.132500
25_120,12240.716833,13266.8500,10240.0,2823.708333,1025.645333


In [208]:
result_group_two = agg_df.groupby('paralell_run').agg(['count','mean', 'min','max'])
result_group_two = result_group_two.sort_values(by=('duration', 'mean'), ascending=True)
result_group_two

duration                                   billed duration  \
                count          mean       min       max           count   
paralell_run                                                              
25_40              40  11268.680000   9961.69  14344.21              40   
25_10              10  11480.164000  10250.95  12606.91              10   
25_80              80  11666.122375   9695.69  14547.75              80   
25_120            120  12240.716833   9741.73  22077.12             120   

                                           memory size                    \
                    mean      min      max       count     mean      min   
paralell_run                                                               
25_40         12113.9750  10593.0  15347.0          40  10240.0  10240.0   
25_10         12355.2000  10951.0  13836.0          10  10240.0  10240.0   
25_80         12644.7625  10233.0  15679.0          80  10240.0  10240.0   
25_120        13266.8500  10460.0  23191.0         120  10240.0  10240.0   

                      max memory used                               \
                  max           count         mean     min     max   
paralell_run                                                         
25_40         10240.0              40  2823.475000  2506.0  2885.0   
25_10         10240.0              10  2841.000000  2812.0  2866.0   
25_80         10240.0              80  2803.412500  2454.0  2887.0   
25_120        10240.0             120  2823.708333  2528.0  2886.0   

             init duration                                
                     count         mean     min      max  
paralell_run                                              
25_40                   40   844.840250  602.60  1061.74  
25_10                   10   874.620000  686.71  1228.99  
25_80                   80   978.132500  536.45  1371.18  
25_120                 120  1025.645333  650.58  1555.15

In [210]:
result_group.to_csv('cosmic_par_results_one.csv')
result_group_two.to_csv('cosmic_par_results_two.csv')